# TFRecord Builder
csv 파일은 문제 없지만, 이미지 파일은 jpg, png 등 용량이 큰 파일로 불러야 하기에, 매번 data를 RAM에 넣어 놓으면 부담

In [1]:
import glob
import os
import tensorflow as tf
import cv2

In [2]:
DATASET_OK_PATTERN = 'data/3/OK/*.png'
DATASET_FAIL_PATTERN = 'data/3/FAIL/*.png'

TFRECORD_PATH = 'tfrecords/'
IMAGE_PER_TERECORD = 100

In [4]:
ok_list = glob.glob(DATASET_OK_PATTERN)
fail_list = glob.glob(DATASET_FAIL_PATTERN)

num_ok = len(ok_list)
num_fail = len(fail_list)

# Oversampling
fail_list_new = list()
for _ in range(num_ok // num_fail):
    fail_list_new += fail_list
fail_list_new += fail_list[: num_ok % num_fail]
fail_list = fail_list_new

ok_label = [0 for _ in range(len(ok_list))]
fail_label = [1 for _ in range(len(fail_list))]

file_list = ok_list + fail_list
label_list = ok_label + fail_label

## TFRecord functions
깃허브에서 가져온 코드
다른 프로젝트에서 사용하면 편리함

In [6]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

# image 저장 시 label은 0과 1 두 개의 숫자, image_raw에는 사진 저장 시 확장자
def image_example(image_string, label):
    image_shape = tf.image.decode_image(image_string).shape

    feature = {
        'height': _int64_feature(image_shape[0]),
        'width': _int64_feature(image_shape[1]),
        'depth': _int64_feature(image_shape[2]),
        'label': _int64_feature(label),
        'image_raw': _bytes_feature(image_string),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

In [8]:
if os.path.exists(TFRECORD_PATH) is False:
    os.mkdir(TFRECORD_PATH)

num_tfrecords = len(file_list) // IMAGE_PER_TERECORD
if len(file_list) % IMAGE_PER_TERECORD != 0:
    num_tfrecords += 1

for idx in range(num_tfrecords):
    idx0 = idx * IMAGE_PER_TERECORD
    idx1 = idx0 + IMAGE_PER_TERECORD
    record_file = TFRECORD_PATH + '%05d.tfrecords' % idx
    with tf.io.TFRecordWriter(record_file) as writer:
        for filename, label in zip(file_list[idx0:idx1], label_list[idx0:idx1]):
            image_string = open(filename, 'rb').read()
            tf_example = image_example(image_string, label)
            writer.write(tf_example.SerializeToString())